# Using Chempath

Chempath is a pathway analysis program that automatically builds the the most important pathways of a reaction system. This jupyter notebook explains how to use chempath. To use Chempath we first need to import it:

In [4]:
from chempath import Chempath, get_latex_contribution_table
import numpy as np

Chempath requires four inputs from a chemical kinetics model in at two consecutive time steps: the reaction system with $n_r$ reactions between $n_s$ species, the species concentrations, the reaction rates, and the model time. The user can also input a minimum rate of pathways $f_{min}$. These input files need to be in a specific format. The folder `input` includes some examples of Chempath input files. The units can be any type of concentration, time and rate units as long as they are consistent. Lets explore these files:

Chempath needs:

- a reaction file with a list of reactions:

In [5]:
input_folder = 'input/simple_HOx/'
# read reactions file
reactions = np.loadtxt(f'{input_folder}/reactions.txt', dtype=str, delimiter='\n')
print('Reactions: ', reactions)


Reactions:  ['H2O2+OH=HO2+H2O' 'OH+OH=H2O2' 'H2O2+O=OH+HO2' 'H2O+HV=OH+O'
 'H2O2+HV=OH+OH']


- a rates file with a rate for each reaction:

In [6]:
# read rates file
rates = np.fromfile(f'{input_folder}/rates.dat', dtype=np.float128)
print('Rates (ppb/hr): ',rates)

Rates (ppb/hr):  [1.  0.5 1.5 5.  0.1]


- a species file with the species names:

In [7]:
# read species file
species = np.loadtxt(f'{input_folder}/species.txt', dtype=str, delimiter='\n')
print('Species: ',species)


Species:  ['H2O2' 'OH' 'HO2' 'H2O' 'O']


- a model time file with two consecutive model times

In [8]:
# read model time file
model_time = np.fromfile(f'{input_folder}/model_time.dat', dtype=np.float128)
print('Model time (hr): ',model_time)

Model time (hr):  [0. 1.]


- a concentrations file with the concentrations of the species at each model time:

In [9]:
# read species concentrations file
concentrations = np.fromfile(f'{input_folder}/concentrations.dat', dtype=np.float128)
print('Concentrations (ppb): ',concentrations.reshape(2, len(species)))

Concentrations (ppb):  [[3.000e+00 6.000e+00 2.000e+00 1.000e+04 1.000e+01]
 [9.000e-01 1.070e+01 4.500e+00 9.996e+03 1.350e+01]]


To create a chempath object we need to provide the files with these inputs:

In [10]:
input_folder = 'input/simple_ozone/'
chempath = Chempath(
        reactions_path=f'{input_folder}/reactions.txt', # file with reaction equations
        rates_path=f'{input_folder}/rates.dat', # file with reaction rates
        species_path=f'{input_folder}/species.txt', # file with species names
        conc_path=f'{input_folder}/concentrations.dat', # file concentrations of species
        time_path=f'{input_folder}/model_time.dat', # file with model time
        f_min=0, # minimum rate of pathways
        dtype=np.float128, # data type of rates and concentrations,
        ignored_sb = ['O2'] # ignore O2 as a branching-point species
    )

Chempath has several attributes that are used to construct pathways. Here is a list of them:

- `chempath.f_min`: minimum rates of pathways
- `chempath.time`: model time
- `chempath.dt`: model time step
- `chempath.mean_time`: mean model time
- `chempath.species_list`: list of species in the reaction system
- `chempath.conc`: species concentrations at times defined in model time
- `chempath.dconc`: species concentration change in model time step 
- `chempath.mean_conc`: species mean concentration
- `chempath.mean_dconc`: species rate of concentration change
- `chempath.reaction_equations`: list of reaction equations
- `chempath.rj`: reaction rates
- `chempath.rj_del`: part of rate of reaction j associated with deleted pathways
- `chempath.sij`: number of molecules of species i produces or destroyed by reaction j
- `chempath.xjk`: multiplicity of reaction j in pathway k
- `chempath.pathway_ids`: list of unique pathways ids
- `chempath.mik`: number of molecules of species i produces or destroyed by pathway k
- `chempath.fk`: rates of pathways
- `chempath.pi_del`: rate of production of species i by deleted pathways
- `chempath.di_del`: rate of destruction of species i by deleted pathways
- `chempath.pi`: rate of production of species i by all pathways
- `chempath.di`: rate of destruction of species i by all pathways
- `chempath.sb_list`: list of used branching-point species
- `chempath.species_of_interest`: list of species of interest
- `chempath.ignored_sb`: list of species not considered as branching-points

In this tutorial we are going to analyze a very simple example with 4 reactions representing the chapman O3 production and destruction reactions. This example eas taken from Lehmann (2004). In this particular example the concentration units are ppb, but in general we can use any concentration units.  

We can see the reaction equations and their rates with:

In [11]:
print('Reactions: ',chempath.reaction_equations) 
print('Rates: ', chempath.rj) # units of ppb/hr

Reactions:  ['O3+hv=O+O2' 'O2+hv=O+O' 'O+O2+M=O3' 'O+O3=O2+O2']
Rates:  [80. 10. 99.  1.]


We can see the species involved in this chemical system, their mean concentrations and their concentrations changes with:

In [12]:
print('Species: ', chempath.species_list)
print('Mean concentrations ', chempath.mean_conc) # units of ppb
print('Concentration changes: ', chempath.dconc) # units of ppb


Species:  ['O3', 'O2', 'O']
Mean concentrations  [1.00009000e+05 9.99999865e+07 1.00000000e+02]
Concentration changes:  [ 18. -27.   0.]


To apply chempath to a chemical model, the reactions must exactly explain the concentration changes of all the species. Let's verify that this condition is true in the current example:

In [13]:
# calculate the ppb  produced or destroyed by the reactions
total_reactions_production = np.dot(chempath.sij, chempath.rj) * chempath.dt
print('total reaction production: ', total_reactions_production)
print('production - loss: ', chempath.pi - chempath.di)
print('Concentration changes: ', chempath.dconc) # units of ppb

total reaction production:  [ 18. -27.   0.]
production - loss:  [ 18. -27.   0.]
Concentration changes:  [ 18. -27.   0.]


Initially each reaction is considered as a pathway:

In [14]:
print('Reactions:\n', np.vstack(chempath.reaction_equations))
print('Multiplicities (xjk):\n', chempath.xjk)

Reactions:
 [['O3+hv=O+O2']
 ['O2+hv=O+O']
 ['O+O2+M=O3']
 ['O+O3=O2+O2']]
Multiplicities (xjk):
 [[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]


We can think of a column in the xjk matrix as a pathway. Intitially there is only one reaction in each pathway. For example the first pathway only has the reaction 'O3+hv=O+O2'.

Chempath constructs pathways through the iterative connection of previously formed pathways through
branching-point species:

<img src="plots/pap_algorithm.png" alt="Algorithm" width=50% height=35%/>

We can perform each of the steps in an iteration of this algorithm with:

In [15]:
# Choice of a branching-pointspecies Sb
sb = chempath.get_sb()
print(f'The branching-point specie is: {sb}')
# get idexes of pathways producing and destroying sb
chempath.get_prod_destr_idxs(sb)
# Formation of new pathway
chempath.form_new_pathways()
# Effect of deleted pathways calculation
chempath.calculate_deleted_pathways_effect()
# Calculation of rates explaining concentration changes
chempath.calculate_rates_explaining_conc_change()
# Deletion of old pathways 
chempath.delete_old_pathways()
# Deletion of pathways with a rate < fmin
chempath.delete_insignificant_pathways()
# Formation of elementary pathways and splitting into sub-pathways
chempath.split_into_subpathways(exact_solutions=True)
# check that rates are correctly distributed
chempath.check_rate_distribution()

The branching-point specie is: O


Yuu can get information about each of these methods using the `?` operator. For example:

In [16]:
chempath.get_prod_destr_idxs?

Signature: chempath.get_prod_destr_idxs(sb)
Docstring:
Gets the indexes of the pathways producing and destroying Sb
        
File:      ~/Files/projects/Pathway Analysis/chempath/chempath.py
Type:      method


In this first iteration the branching-point species is O. All reactions producing O will be connected with all reactions consuming O to create new pathways. We can see the new pathways in the matrix xjk:

In [17]:
chempath.xjk

array([[1., 1., 0., 0.],
       [0., 0., 1., 1.],
       [1., 0., 2., 0.],
       [0., 1., 0., 2.]])

The method 'get_pathway_str' converts the representation of a pathways as an array of multiplicities to a string:

In [18]:
chempath.get_pathway_str?

Signature: chempath.get_pathway_str(xjc, format='txt', include_net_reaction=True)
Docstring:
Gets the string of a pathway given its multiplicities.
Arguments:
    xjc (numpy 1d array): multiplicities of pathway
    format(str): format of the pathway string. Can be 'txt', 'latex' or
        'html
    include_net_reaction(bool): If true includes the net reaction in 
        the pathway string
Returns:
    pathway string (str)
File:      ~/Files/projects/Pathway Analysis/chempath/chempath.py
Type:      method


We can use this method to see the pathways found by Chempath in this iteration:

In [19]:
# print all pathways formed and their rates
for i in range(len(chempath.fk)):
    print(f'Pathway {i+1} with rate={chempath.fk[i]}ppb:')
    pathway = chempath.get_pathway_str(chempath.xjk[:,i])
    print(pathway)
    print('\n')

Pathway 1 with rate=79.2ppb:
O3+hv -> O+O2
O+O2+M -> O3
Net:Null


Pathway 2 with rate=0.8ppb:
O3+hv -> O+O2
O+O3 -> O2+O2
Net:2O3 -> 3O2


Pathway 3 with rate=9.9ppb:
O2+hv -> O+O
2(O+O2+M -> O3)
Net:3O2 -> 2O3


Pathway 4 with rate=0.1ppb:
O2+hv -> O+O
2(O+O3 -> O2+O2)
Net:2O3 -> 3O2




The pathways found by the algorithm must produce the same number of molecules as the initial reactions. Let's verify that this is the case:

In [20]:
pathway_total_production = np.dot(chempath.mik, chempath.fk) * chempath.dt
print('total pathway production: ', pathway_total_production)

total pathway production:  [ 18. -27.   0.]


We can see the contributions of the pathways to the loss or production of a species with:

In [21]:
chempath.get_pathways_contributions?

Signature: chempath.get_pathways_contributions(sp, on='loss', format='txt')
Docstring:
Calculate the contribution of all pathways to the loss or production 
of a species
Arguments:
    sp(str): species to calculate contributions for
    on(str): can be 'loss' or 'production'
    format(str): format of pathway strings. Can be 'txt', 'html' and 
    'latex'
Returns:
    contrib_df: pandas dataframe with contributions    
File:      ~/Files/projects/Pathway Analysis/chempath/chempath.py
Type:      method


In [22]:
chempath.get_pathways_contributions('O3', 'loss')

,pathway_id,pathway,contribution,rate,total_prod,dconc
0,"1*0,1*3",O3+hv -> O+O2\nO+O3 -> O2+O2\nNet:2O3 -> 3O2,0.888889,0.8,-1.6,18.0
1,"1*1,2*3",O2+hv -> O+O\n2(O+O3 -> O2+O2)\nNet:2O3 -> 3O2,0.111111,0.1,-0.2,18.0
2,del,deleted_pathways,0.000000,0.0,0.0,18.0


In [23]:
chempath.get_pathways_contributions('O2', 'production')

,pathway_id,pathway,contribution,rate,total_prod,dconc
0,"1*0,1*3",O3+hv -> O+O2\nO+O3 -> O2+O2\nNet:2O3 -> 3O2,0.888889,0.8,2.4,-27.0
1,"1*1,2*3",O2+hv -> O+O\n2(O+O3 -> O2+O2)\nNet:2O3 -> 3O2,0.111111,0.1,0.3,-27.0
2,del,deleted_pathways,0.000000,0.0,0.0,-27.0


Let's do another iteration to find new pathways. The method `find_new_pathways` performs the same steps as a above to do an iteration of the algorithm at a specific branching-point species.

In [24]:
sb =chempath.get_sb()
print(f'The branching-point specie is: {sb}')
chempath.find_new_pathways(sb)

# print all pathways formed and their rates
for i in range(len(chempath.fk)):
    print(f'Pathway {i+1} with rate={chempath.fk[i]}ppb:')
    pathway = chempath.get_pathway_str(chempath.xjk[:,i])
    print(pathway)
    print('\n')

The branching-point specie is: O3
Pathway 1 with rate=80.0ppb:
O3+hv -> O+O2
O+O2+M -> O3
Net:Null


Pathway 2 with rate=9.0ppb:
O2+hv -> O+O
2(O+O2+M -> O3)
Net:3O2 -> 2O3


Pathway 3 with rate=1.0000000000000002ppb:
O2+hv -> O+O
O+O2+M -> O3
O+O3 -> O2+O2
Net:Null




We can find new pathways one branching-point species at a time as shown above. Alternatively we can use the `find_all_pathways` method to iterate over all branching-points species:

In [25]:
#re-initialize chempath object
chempath.reinit()

# find all pathways
chempath.find_all_pathways()

# print all pathways formed and their rates
for i in range(len(chempath.fk)):
    print(f'Pathway {i+1} with rate={chempath.fk[i]}ppb:')
    pathway = chempath.get_pathway_str(chempath.xjk[:,i])
    print(pathway)
    print('\n')

Pathway 1 with rate=80.0ppb:
O3+hv -> O+O2
O+O2+M -> O3
Net:Null


Pathway 2 with rate=9.0ppb:
O2+hv -> O+O
2(O+O2+M -> O3)
Net:3O2 -> 2O3


Pathway 3 with rate=1.0000000000000002ppb:
O2+hv -> O+O
O+O2+M -> O3
O+O3 -> O2+O2
Net:Null




The attribute `sb_list` contains the list of barnching point species used by the algorithm in the same order they were used:

In [26]:
chempath.sb_list

['O', 'O3']

# Using $f_{min}$

If we set $f_{min}$ to a value different than zero, the pathways with rates lower than $f_{min}$ will be deleted. For example, we can set $f_min=2ppb$:

In [27]:
#re-initialize chempath object
chempath.reinit()

# set fmin = 2
chempath.f_min = 2

# find all pathways
chempath.find_all_pathways()

chempath.find_all_pathways()

# print all pathways formed and their rates
for i in range(len(chempath.fk)):
    print(f'Pathway {i+1} with rate={chempath.fk[i]}ppb:')
    pathway = chempath.get_pathway_str(chempath.xjk[:,i])
    print(pathway)
    print('\n')

Pathway 1 with rate=79.2ppb:
O3+hv -> O+O2
O+O2+M -> O3
Net:Null


Pathway 2 with rate=9.9ppb:
O2+hv -> O+O
2(O+O2+M -> O3)
Net:3O2 -> 2O3




In this case, the pathway 3 found above was deleted, and the attributes `pi_del` and `di_del` that store the rate production and destruction of species by deleted pathways will have values different than zero:

In [28]:
print('rate of production of species by deleted pathways: ',chempath.pi_del)
print('rate of destruction of species by deleted pathways: ',chempath.di_del)

rate of production of species by deleted pathways:  [0.  2.7 0. ]
rate of destruction of species by deleted pathways:  [1.8 0.  0. ]


In this case, the pathways by themselves no longer explain the concentration changes because some pathways were deleted:

In [29]:
pathway_total_production = np.dot(chempath.mik, chempath.fk) * chempath.dt
print('total pathway production: ', pathway_total_production)
print('Concentration changes: ', chempath.dconc) # units of ppb

total pathway production:  [ 19.8 -29.7   0. ]
Concentration changes:  [ 18. -27.   0.]


However, if we include the effect of deleted pathways, the concentration changes are explained:

In [30]:
pathway_total_production = np.dot(chempath.mik, chempath.fk) * chempath.dt
print('total pathway production: ', pathway_total_production + chempath.pi_del - chempath.di_del)
print('Concentration changes: ', chempath.dconc) # units of ppb

total pathway production:  [ 18. -27.   0.]
Concentration changes:  [ 18. -27.   0.]


# Simple $HO_x$ example 

In the main manuscript we used a simple example to explain the algorithm. We can use Chempath to find the pathways in this simple example:

In [31]:
# initialize chempath
input_folder = 'input/simple_HOx/'
chempath = Chempath(
        reactions_path=f'{input_folder}/reactions.txt', # file with reaction equations
        rates_path=f'{input_folder}/rates.dat', # file with reaction rates
        species_path=f'{input_folder}/species.txt', # file with species names
        conc_path=f'{input_folder}/concentrations.dat', # file concentrations of species
        time_path=f'{input_folder}/model_time.dat', # file with model time
        f_min=0.02, # minimum rate of pathways
        dtype=np.float128, # data type of rates and concentrations,
        ignored_sb = ['HO2', 'H2O'] # ignore HO2 and H2O as a branching-point species
    )

# finad all pathways
chempath.find_all_pathways()

# get contributions of pathways producing HO2
chempath.get_pathways_contributions('HO2', on='production')

,pathway_id,pathway,contribution,rate,total_prod,dconc
0,"1*2,1*3",H2O2+O -> OH+HO2\nH2O+HV -> OH+O\nNet:H2O2 + H...,0.374336,0.935841,0.935841,2.5
1,"2*0,1*2,1*3",2(H2O2+OH -> HO2+H2O)\nH2O2+O -> OH+HO2\nH2O+H...,0.206493,0.172078,0.516233,2.5
2,"1*0,1*3",H2O2+OH -> HO2+H2O\nH2O+HV -> OH+O\nNet:H2O2 -...,0.185841,0.464602,0.464602,2.5
3,"1*1,1*2,1*3",OH+OH -> H2O2\nH2O2+O -> OH+HO2\nH2O+HV -> OH+...,0.130060,0.325149,0.325149,2.5
4,del,deleted_pathways,0.059022,0.147556,0.147556,2.5
5,"1*0,1*1,3*3",H2O2+OH -> HO2+H2O\nOH+OH -> H2O2\n3(H2O+HV ->...,0.044248,0.110619,0.110619,2.5


We can get a latex table with these contributions:

In [32]:
# get contributions of pathways producing HO2
contribution_df = chempath.get_pathways_contributions('HO2', on='production',
    format='latex')
print(get_latex_contribution_table(contribution_df))



        \begin{longtable}{ |c|c|c|c| }
        \hline
        ID & Pathway & Contribution (\%) & Rate \\
        \hline
        1 & \begin{tabular}{@{}c@{}}\ce{ H2O2 + O -> OH + HO2\\H2O + HV -> OH + O\\\text{Net:} H2O2 + H2O -> 2OH + HO2 }\end{tabular} & 37.434 & 0.936\\
 \hline 
2 & \begin{tabular}{@{}c@{}}\ce{ 2(H2O2 + OH -> HO2 + H2O)\\H2O2 + O -> OH + HO2\\H2O + HV -> OH + O\\\text{Net:} 3H2O2 -> 3HO2 + H2O }\end{tabular} & 20.649 & 0.172\\
 \hline 
3 & \begin{tabular}{@{}c@{}}\ce{ H2O2 + OH -> HO2 + H2O\\H2O + HV -> OH + O\\\text{Net:} H2O2 -> HO2 + O }\end{tabular} & 18.584 & 0.465\\
 \hline 
4 & \begin{tabular}{@{}c@{}}\ce{ OH + OH -> H2O2\\H2O2 + O -> OH + HO2\\H2O + HV -> OH + O\\\text{Net:} H2O -> HO2 }\end{tabular} & 13.006 & 0.325\\
 \hline 
5 & \begin{tabular}{@{}c@{}}deleted_pathways\end{tabular} & 5.902 & 0.148\\
 \hline 

        \end{longtable}
        


You can also use the pandas `to_latex` and `to_markdown` methods to make tables:

In [33]:
print(contribution_df.to_latex())

\begin{tabular}{lllrrrr}
\toprule
{} &   pathway\_id &                                            pathway &  contribution &      rate &  total\_prod &  dconc \\
\midrule
0 &      1*2,1*3 &  \textbackslash ce\{ H2O2 + O -> OH + HO2\textbackslash \textbackslash H2O + HV -> OH + O\textbackslash ... &      0.374336 &  0.935841 &    0.935841 &    2.5 \\
1 &  2*0,1*2,1*3 &  \textbackslash ce\{ 2(H2O2 + OH -> HO2 + H2O)\textbackslash \textbackslash H2O2 + O -> OH... &      0.206493 &  0.172078 &    0.516233 &    2.5 \\
2 &      1*0,1*3 &  \textbackslash ce\{ H2O2 + OH -> HO2 + H2O\textbackslash \textbackslash H2O + HV -> OH + ... &      0.185841 &  0.464602 &    0.464602 &    2.5 \\
3 &  1*1,1*2,1*3 &  \textbackslash ce\{ OH + OH -> H2O2\textbackslash \textbackslash H2O2 + O -> OH + HO2\textbackslash \textbackslash H2... &      0.130060 &  0.325149 &    0.325149 &    2.5 \\
4 &          del &                                   deleted\_pathways &      0.059022 &  0.147556 &    0.147556 &    2.

In [34]:
contribution_df = chempath.get_pathways_contributions('HO2', on='production',
    format='txt')
print(contribution_df[['pathway', 'contribution', 'rate']].to_markdown())

|    | pathway                     |   contribution |     rate |
|---:|:----------------------------|---------------:|---------:|
|  0 | H2O2+O -> OH+HO2            |      0.374336  | 0.935841 |
|    | H2O+HV -> OH+O              |                |          |
|    | Net:H2O2 + H2O -> 2OH + HO2 |                |          |
|  1 | 2(H2O2+OH -> HO2+H2O)       |      0.206493  | 0.172078 |
|    | H2O2+O -> OH+HO2            |                |          |
|    | H2O+HV -> OH+O              |                |          |
|    | Net:3H2O2 -> 3HO2 + H2O     |                |          |
|  2 | H2O2+OH -> HO2+H2O          |      0.185841  | 0.464602 |
|    | H2O+HV -> OH+O              |                |          |
|    | Net:H2O2 -> HO2 + O         |                |          |
|  3 | OH+OH -> H2O2               |      0.13006   | 0.325149 |
|    | H2O2+O -> OH+HO2            |                |          |
|    | H2O+HV -> OH+O              |                |          |
|    | Net:H2O -> HO2    